# Import libraries

In [1]:
import pandas as pd 
import os 
import requests 
import sqlalchemy
import time
from dotenv import load_dotenv 
load_dotenv()

True

# First test

In [2]:

API_KEY = os.getenv("API_KEY")

url = "https://api.eia.gov/v2/nuclear-outages/generator-nuclear-outages/data/"

params = {
    "frequency": "daily",
    "data[0]": "capacity",
    "data[1]": "outage",
    "data[2]": "percentOutage",
    "sort[0][column]": "period",
    "sort[0][direction]": "desc",
    "offset": 0,
    #"length": 500,
    "api_key": API_KEY
}

response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()
else:
    print(f"Error: {response.status_code}")
    print(response.text)


In [3]:
data_list = data['response']['data']
df = pd.DataFrame(data_list)
df.head()

,period,facility,facilityName,generator,capacity,outage,percentOutage,capacity-units,outage-units,percentOutage-units
0,2025-09-03,6040,Beaver Valley,1,907,0,0,megawatts,megawatts,percent
1,2025-09-03,6251,South Texas Project,2,1280,0,0,megawatts,megawatts,percent
2,2025-09-03,880,Quad Cities Generating Station,1,908,0,0,megawatts,megawatts,percent
3,2025-09-03,6011,Calvert Cliffs Nuclear Power Plant,1,884.2,0,0,megawatts,megawatts,percent
4,2025-09-03,6145,Comanche Peak,2,1195,0,0,megawatts,megawatts,percent


In [7]:
df["period"] = pd.to_datetime(df["period"])
df["capacity"] = pd.to_numeric(df["capacity"])
df["outage"] = pd.to_numeric(df["outage"])
df["percentOutage"] = pd.to_numeric(df["percentOutage"])

In [17]:
df.columns

Index(['period', 'facility', 'facilityName', 'generator', 'capacity', 'outage',
       'percentOutage', 'capacity-units', 'outage-units',
       'percentOutage-units'],
      dtype='object')

In [28]:
df.isna().sum()

period           0
facility         0
facilityName     0
generator        0
capacity         0
outage           0
percentOutage    0
dtype: int64

In [18]:
df.drop(columns=["capacity-units","outage-units","percentOutage-units"], inplace=True)

# Creation of the database

### Connection to the database

In [2]:
server   = "eia-server.database.windows.net"  
database = "eiaDB"               # 
username = "mathis"              
password = os.getenv("PASSWORD_EIADB")                    
driver   = "ODBC Driver 17 for SQL Server"

connection_string = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver.replace(' ', '+')}"
engine = sqlalchemy.create_engine(connection_string)


### Test read_sql

In [9]:
QUERY = """
    SELECT TOP 1 period
    FROM Outages"""

df = pd.read_sql(QUERY, con=engine)

In [3]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT TOP 5 period
        FROM Outages
        ORDER BY period DESC
    """))
    last_period = result.scalar()   # fetch the first column of the first row

print(last_period)

2025-09-02


### Test of the connection

In [4]:
with engine.connect() as conn:
    result = conn.execute(sqlalchemy.text("SELECT @@VERSION"))
    for row in result:
        print(row)

('Microsoft SQL Azure (RTM) - 12.0.2000.8 \n\tAug  6 2025 10:21:16 \n\tCopyright (C) 2025 Microsoft Corporation\n',)


# Function to retrieve data

In [ ]:
def retrieve_and_clean_data_from_api(API_KEY: str, start_date: str, end_date: str) -> pd.DataFrame:
    """
    Retrieve and clean data from the EIA API for a given date range.

    Parameters:
    API_KEY (str): EIA API key.
    start_date (str): The start date in 'YYYY-MM-DD' format.
    end_date (str): The end date in 'YYYY-MM-DD' format.


    Returns:
    pd.DataFrame: Cleaned DataFrame containing the data.
    """

    # Url and params to do the request to the API
    url = "https://api.eia.gov/v2/nuclear-outages/generator-nuclear-outages/data/"

    params = {
        "frequency": "daily",
        "data[0]": "capacity",
        "data[1]": "outage",
        "data[2]": "percentOutage",
        "start": start_date,
        "end": end_date,
        "sort[0][column]": "period",
        "sort[0][direction]": "desc",
        "offset": 0,
        "api_key": API_KEY
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        
        # We retrieve the data 
        data_list = data['response']['data']

        # Convert to pandas DataFrame
        df = pd.DataFrame(data_list)

        # We delete the columns that are useless 
        df.drop(columns=["capacity-units","outage-units","percentOutage-units"], inplace=True)

        # We convert the columns to the right type
        df["period"] = pd.to_datetime(df["period"])
        df["capacity"] = pd.to_numeric(df["capacity"])
        df["outage"] = pd.to_numeric(df["outage"])
        df["percentOutage"] = pd.to_numeric(df["percentOutage"])

        return df
    
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return pd.DataFrame()
    
    

# Push it on the table

In [9]:
def to_sql(df: pd.DataFrame, table_name: str, engine: sqlalchemy.engine.base.Engine) -> None:
    """
    Push a DataFrame to a SQL table.

    Parameters:
    df (pd.DataFrame): The DataFrame to push.
    table_name (str): The name of the target SQL table.
    engine (sqlalchemy.engine.base.Engine): SQLAlchemy engine connected to the database.

    Returns:
    None
    """
    df.to_sql(table_name, con=engine, if_exists="append", index=False)

# Retrieve all the data

We retrieve the data month by month 

In [11]:
month_ranges = [
    ("2007-01-01", "2007-01-31"),
    ("2007-02-01", "2007-02-28"),
    ("2007-03-01", "2007-03-31"),
    ("2007-04-01", "2007-04-30"),
    ("2007-05-01", "2007-05-31"),
    ("2007-06-01", "2007-06-30"),
    ("2007-07-01", "2007-07-31"),
    ("2007-08-01", "2007-08-31"),
    ("2007-09-01", "2007-09-30"),
    ("2007-10-01", "2007-10-31"),
    ("2007-11-01", "2007-11-30"),
    ("2007-12-01", "2007-12-31"),
    ("2008-01-01", "2008-01-31"),
    ("2008-02-01", "2008-02-29"),
    ("2008-03-01", "2008-03-31"),
    ("2008-04-01", "2008-04-30"),
    ("2008-05-01", "2008-05-31"),
    ("2008-06-01", "2008-06-30"),
    ("2008-07-01", "2008-07-31"),
    ("2008-08-01", "2008-08-31"),
    ("2008-09-01", "2008-09-30"),
    ("2008-10-01", "2008-10-31"),
    ("2008-11-01", "2008-11-30"),
    ("2008-12-01", "2008-12-31"),
    ("2009-01-01", "2009-01-31"),
    ("2009-02-01", "2009-02-28"),
    ("2009-03-01", "2009-03-31"),
    ("2009-04-01", "2009-04-30"),
    ("2009-05-01", "2009-05-31"),
    ("2009-06-01", "2009-06-30"),
    ("2009-07-01", "2009-07-31"),
    ("2009-08-01", "2009-08-31"),
    ("2009-09-01", "2009-09-30"),
    ("2009-10-01", "2009-10-31"),
    ("2009-11-01", "2009-11-30"),
    ("2009-12-01", "2009-12-31"),
    ("2010-01-01", "2010-01-31"),
    ("2010-02-01", "2010-02-28"),
    ("2010-03-01", "2010-03-31"),
    ("2010-04-01", "2010-04-30"),
    ("2010-05-01", "2010-05-31"),
    ("2010-06-01", "2010-06-30"),
    ("2010-07-01", "2010-07-31"),
    ("2010-08-01", "2010-08-31"),
    ("2010-09-01", "2010-09-30"),
    ("2010-10-01", "2010-10-31"),
    ("2010-11-01", "2010-11-30"),
    ("2010-12-01", "2010-12-31"),
    ("2011-01-01", "2011-01-31"),
    ("2011-02-01", "2011-02-28"),
    ("2011-03-01", "2011-03-31"),
    ("2011-04-01", "2011-04-30"),
    ("2011-05-01", "2011-05-31"),
    ("2011-06-01", "2011-06-30"),
    ("2011-07-01", "2011-07-31"),
    ("2011-08-01", "2011-08-31"),
    ("2011-09-01", "2011-09-30"),
    ("2011-10-01", "2011-10-31"),
    ("2011-11-01", "2011-11-30"),
    ("2011-12-01", "2011-12-31"),
    ("2012-01-01", "2012-01-31"),
    ("2012-02-01", "2012-02-29"),
    ("2012-03-01", "2012-03-31"),
    ("2012-04-01", "2012-04-30"),
    ("2012-05-01", "2012-05-31"),
    ("2012-06-01", "2012-06-30"),
    ("2012-07-01", "2012-07-31"),
    ("2012-08-01", "2012-08-31"),
    ("2012-09-01", "2012-09-30"),
    ("2012-10-01", "2012-10-31"),
    ("2012-11-01", "2012-11-30"),
    ("2012-12-01", "2012-12-31"),
    ("2013-01-01", "2013-01-31"),
    ("2013-02-01", "2013-02-28"),
    ("2013-03-01", "2013-03-31"),
    ("2013-04-01", "2013-04-30"),
    ("2013-05-01", "2013-05-31"),
    ("2013-06-01", "2013-06-30"),
    ("2013-07-01", "2013-07-31"),
    ("2013-08-01", "2013-08-31"),
    ("2013-09-01", "2013-09-30"),
    ("2013-10-01", "2013-10-31"),
    ("2013-11-01", "2013-11-30"),
    ("2013-12-01", "2013-12-31"),
    ("2014-01-01", "2014-01-31"),
    ("2014-02-01", "2014-02-28"),
    ("2014-03-01", "2014-03-31"),
    ("2014-04-01", "2014-04-30"),
    ("2014-05-01", "2014-05-31"),
    ("2014-06-01", "2014-06-30"),
    ("2014-07-01", "2014-07-31"),
    ("2014-08-01", "2014-08-31"),
    ("2014-09-01", "2014-09-30"),
    ("2014-10-01", "2014-10-31"),
    ("2014-11-01", "2014-11-30"),
    ("2014-12-01", "2014-12-31"),
    ("2015-01-01", "2015-01-31"),
    ("2015-02-01", "2015-02-28"),
    ("2015-03-01", "2015-03-31"),
    ("2015-04-01", "2015-04-30"),
    ("2015-05-01", "2015-05-31"),
    ("2015-06-01", "2015-06-30"),
    ("2015-07-01", "2015-07-31"),
    ("2015-08-01", "2015-08-31"),
    ("2015-09-01", "2015-09-30"),
    ("2015-10-01", "2015-10-31"),
    ("2015-11-01", "2015-11-30"),
    ("2015-12-01", "2015-12-31"),
    ("2016-01-01", "2016-01-31"),
    ("2016-02-01", "2016-02-29"),
    ("2016-03-01", "2016-03-31"),
    ("2016-04-01", "2016-04-30"),
    ("2016-05-01", "2016-05-31"),
    ("2016-06-01", "2016-06-30"),
    ("2016-07-01", "2016-07-31"),
    ("2016-08-01", "2016-08-31"),
    ("2016-09-01", "2016-09-30"),
    ("2016-10-01", "2016-10-31"),
    ("2016-11-01", "2016-11-30"),
    ("2016-12-01", "2016-12-31"),
    ("2017-01-01", "2017-01-31"),
    ("2017-02-01", "2017-02-28"),
    ("2017-03-01", "2017-03-31"),
    ("2017-04-01", "2017-04-30"),
    ("2017-05-01", "2017-05-31"),
    ("2017-06-01", "2017-06-30"),
    ("2017-07-01", "2017-07-31"),
    ("2017-08-01", "2017-08-31"),
    ("2017-09-01", "2017-09-30"),
    ("2017-10-01", "2017-10-31"),
    ("2017-11-01", "2017-11-30"),
    ("2017-12-01", "2017-12-31"),
    ("2018-01-01", "2018-01-31"),
    ("2018-02-01", "2018-02-28"),
    ("2018-03-01", "2018-03-31"),
    ("2018-04-01", "2018-04-30"),
    ("2018-05-01", "2018-05-31"),
    ("2018-06-01", "2018-06-30"),
    ("2018-07-01", "2018-07-31"),
    ("2018-08-01", "2018-08-31"),
    ("2018-09-01", "2018-09-30"),
    ("2018-10-01", "2018-10-31"),
    ("2018-11-01", "2018-11-30"),
    ("2018-12-01", "2018-12-31"),
    ("2019-01-01", "2019-01-31"),
    ("2019-02-01", "2019-02-28"),
    ("2019-03-01", "2019-03-31"),
    ("2019-04-01", "2019-04-30"),
    ("2019-05-01", "2019-05-31"),
    ("2019-06-01", "2019-06-30"),
    ("2019-07-01", "2019-07-31"),
    ("2019-08-01", "2019-08-31"),
    ("2019-09-01", "2019-09-30"),
    ("2019-10-01", "2019-10-31"),
    ("2019-11-01", "2019-11-30"),
    ("2019-12-01", "2019-12-31"),
    ("2020-01-01", "2020-01-31"),
    ("2020-02-01", "2020-02-29"),
    ("2020-03-01", "2020-03-31"),
    ("2020-04-01", "2020-04-30"),
    ("2020-05-01", "2020-05-31"),
    ("2020-06-01", "2020-06-30"),
    ("2020-07-01", "2020-07-31"),
    ("2020-08-01", "2020-08-31"),
    ("2020-09-01", "2020-09-30"),
    ("2020-10-01", "2020-10-31"),
    ("2020-11-01", "2020-11-30"),
    ("2020-12-01", "2020-12-31"),
    ("2021-01-01", "2021-01-31"),
    ("2021-02-01", "2021-02-28"),
    ("2021-03-01", "2021-03-31"),
    ("2021-04-01", "2021-04-30"),
    ("2021-05-01", "2021-05-31"),
    ("2021-06-01", "2021-06-30"),
    ("2021-07-01", "2021-07-31"),
    ("2021-08-01", "2021-08-31"),
    ("2021-09-01", "2021-09-30"),
    ("2021-10-01", "2021-10-31"),
    ("2021-11-01", "2021-11-30"),
    ("2021-12-01", "2021-12-31"),
    ("2022-01-01", "2022-01-31"),
    ("2022-02-01", "2022-02-28"),
    ("2022-03-01", "2022-03-31"),
    ("2022-04-01", "2022-04-30"),
    ("2022-05-01", "2022-05-31"),
    ("2022-06-01", "2022-06-30"),
    ("2022-07-01", "2022-07-31"),
    ("2022-08-01", "2022-08-31"),
    ("2022-09-01", "2022-09-30"),
    ("2022-10-01", "2022-10-31"),
    ("2022-11-01", "2022-11-30"),
    ("2022-12-01", "2022-12-31"),
    ("2023-01-01", "2023-01-31"),
    ("2023-02-01", "2023-02-28"),
    ("2023-03-01", "2023-03-31"),
    ("2023-04-01", "2023-04-30"),
    ("2023-05-01", "2023-05-31"),
    ("2023-06-01", "2023-06-30"),
    ("2023-07-01", "2023-07-31"),
    ("2023-08-01", "2023-08-31"),
    ("2023-09-01", "2023-09-30"),
    ("2023-10-01", "2023-10-31"),
    ("2023-11-01", "2023-11-30"),
    ("2023-12-01", "2023-12-31"),
    ("2024-01-01", "2024-01-31"),
    ("2024-02-01", "2024-02-29"),
    ("2024-03-01", "2024-03-31"),
    ("2024-04-01", "2024-04-30"),
    ("2024-05-01", "2024-05-31"),
    ("2024-06-01", "2024-06-30"),
    ("2024-07-01", "2024-07-31"),
    ("2024-08-01", "2024-08-31"),
    ("2024-09-01", "2024-09-30"),
    ("2024-10-01", "2024-10-31"),
    ("2024-11-01", "2024-11-30"),
    ("2024-12-01", "2024-12-31"),
    ("2025-01-01", "2025-01-31"),
    ("2025-02-01", "2025-02-28"),
    ("2025-03-01", "2025-03-31"),
    ("2025-04-01", "2025-04-30"),
    ("2025-05-01", "2025-05-31"),
    ("2025-06-01", "2025-06-30"),
    ("2025-07-01", "2025-07-31"),
    ("2025-08-01", "2025-08-31"),
    ("2025-09-01", "2025-09-02"),
]


In [33]:
month_ranges[0][0]

'2007-01-01'

In [ ]:
for date in month_ranges : 
    # We add a delay to avoid overloading the API
    time.sleep(5)
    start_date = date[0]
    end_date = date[1]
    # We retrieve the data month by month 
    df = retrieve_and_clean_data_from_api(API_KEY, start_date, end_date)
    
    # If the dataframe is not empty we push it to the database
    if len(df) != 0 : 
        to_sql(df, "Outages", engine)
        print(f"Data from {start_date} to {end_date} pushed to the database.")
    else :
        print(f"No data from {start_date} to {end_date}.")

Data from 2007-09-01 to 2007-09-30 pushed to the database.
Data from 2007-10-01 to 2007-10-31 pushed to the database.
Data from 2007-11-01 to 2007-11-30 pushed to the database.
Data from 2007-12-01 to 2007-12-31 pushed to the database.
Data from 2008-01-01 to 2008-01-31 pushed to the database.
Data from 2008-02-01 to 2008-02-29 pushed to the database.
Data from 2008-03-01 to 2008-03-31 pushed to the database.
Data from 2008-04-01 to 2008-04-30 pushed to the database.
Data from 2008-05-01 to 2008-05-31 pushed to the database.
Data from 2008-06-01 to 2008-06-30 pushed to the database.
Data from 2008-07-01 to 2008-07-31 pushed to the database.
Data from 2008-08-01 to 2008-08-31 pushed to the database.
Data from 2008-09-01 to 2008-09-30 pushed to the database.
Data from 2008-10-01 to 2008-10-31 pushed to the database.
Data from 2008-11-01 to 2008-11-30 pushed to the database.
Data from 2008-12-01 to 2008-12-31 pushed to the database.
Data from 2009-01-01 to 2009-01-31 pushed to the databas